In [1]:
import pandas as pd
import numpy as np
# import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

# 전체 컬럼, 전체 행, 전체 숫자 출력
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:f}'.format)

# # plot 출력
# %matplotlib inline

# # plot 한글출력
# import matplotlib.font_manager as fm

# for font in fm.fontManager.ttflist:
#     if font.name in ['AppleGothic', 'NanumGothic']:
#         plt.rcParams['font.family'] = font.name
#     break

# warning message 제거
import warnings
warnings.filterwarnings('ignore')

In [2]:
boxoffice = pd.read_csv('films_updated.csv')
print(boxoffice.shape)
boxoffice.sample()

(686, 38)


,영화명,영화코드,개봉일,대표국적,국적,장르,등급,영화형태,영화구분,제작사,배급사,상영횟수,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,F-rated,감독,대표감독,감독코드,감독성별,감독_대표역할,감독필모,작가,대표작가,대표작가(필명),작가코드,작가성별,작가_대표역할,작가필모,배우,주연배우,배역명,배우코드,배우성별,배우_대표역할,배우필모
95,탐정 : 더 비기닝,20156557,2015-09-24,한국,한국,"코미디,범죄",15세이상관람가,장편,일반영화,(주)크리픽쳐스,씨제이이앤엠(주),54995,764,20463956973.000000,2625686,4092941173,504961,1,김정훈,김정훈,10005643,남자,감독,탐정 : 더 비기닝|쩨쩨한 로맨스|어깨동무,"황혜순, 김정훈",황혜순,황혜순,20110688,여자,시나리오(각본),탐정 : 더 비기닝|쩨쩨한 로맨스,"권상우,성동일,박해준,이승준,조복래,오정세,박준면,이일화,서영희,강민지,홍서현,천유...",권상우,강대만,10002387,남자,배우,신의 한 수: 귀수편|두번할까요|히트맨|탐정: 리턴즈|탐정 : 더 비기닝|그림자 애...


In [3]:
k_movie_list = list(boxoffice['영화명'])

len(k_movie_list)

686

In [136]:
# df.iloc[:, 2:-1] = df.iloc[:, 2:-1].astype(str).applymap(lambda x: x if x.split(',')[1].split('(')[0].strip() in ko_movie_list else '-')
df.iloc[:, 2:-1].astype(str).applymap(lambda x: x if x.split(', ')[1].split('(')[0].strip() in ko_movie_list else '-')

# showtimes = [''.join(df.iloc[i, 2:-1]) for i in range(len(df))]
# df['상영시간'] = showtimes

IndexError: ('list index out of range', 'occurred at index 1회')

# 상영내역 파일 확인

In [125]:
df = pd.read_csv('Theaters_merged_files/CGV%20판교.csv')
print(df.shape)
df.sample()

(10743, 14)


,상영일자,상영관,1회,2회,3회,4회,5회,6회,7회,8회,9회,10회,11회,좌석수
10166,2019-08-10,IMAX,"06:35, 봉오동 전투(디지털), 8,000원","09:15, 봉오동 전투(디지털), 8,000원","11:55, 봉오동 전투(디지털), 13,000원","14:35, 봉오동 전투(디지털), 13,000원","17:15, 봉오동 전투(디지털), 13,000원","19:55, 봉오동 전투(디지털), 13,000원","22:35, 봉오동 전투(디지털), 13,000원","25:15, 봉오동 전투(디지털), 13,000원",NaN,NaN,NaN,297


In [15]:
theaters = pd.read_csv('Theaters_crawled_files/theaters_191128.csv')
theater_list = theaters['영화상영관명']

In [16]:
theater_list.sample()

451    완주 휴 시네마
Name: 영화상영관명, dtype: object

In [8]:
# 파일 수정
# cgv 압구정 165
# cgv 신촌아트레온, 15
# 롯데시네마 광주(롯데백화점), 432
# 롯데시네마 광주(롯데백화점)_샤롯데 461
theaters[theaters['영화상영관명'] == '롯데시네마 광주(롯데백화점)_샤롯데']

,광역단체,기초단체,영화상영관코드,영화상영관명,총 스크린수,총 좌석수,필름 상영관수,2D 상영관수,3D 상영관수,4D 상영관수,IMAX 상영관수,상설여부,특별관운영여부,가입여부,전송사업자명,개관일,영업상태,운영형태,주소,전화번호,홈페이지
461,광주시,동구,16037,롯데시네마 광주(롯데백화점)_샤롯데,1,30,0,1,0,0,0,상설,nan,가입,롯데컬처웍스(주)롯데시네마,2000-07-01,영업,직영,광주광역시 동구 대인동 7-1 번지,1544-8855,http://www.lottecinema.co.kr/


In [172]:
test = pd.read_csv('Theaters_merged_files/강릉자동차극장.csv'
# #                    , encoding='utf-8'
                  )
print(test.shape)
test.sample()

(1, 11)


,상영일자,상영관,1회,2회,3회,4회,5회,6회,7회,8회,좌석수
0,검색된 데이터가 존재하지 않습니다.,검색된 데이터가 존재하지 않습니다.,검색된 데이터가 존재하지 않습니다.,검색된 데이터가 존재하지 않습니다.,검색된 데이터가 존재하지 않습니다.,검색된 데이터가 존재하지 않습니다.,검색된 데이터가 존재하지 않습니다.,검색된 데이터가 존재하지 않습니다.,검색된 데이터가 존재하지 않습니다.,검색된 데이터가 존재하지 않습니다.,nan


In [133]:
list(test.columns)[2:-1]

['1회', '2회', '3회', '4회', '5회', '6회', '7회', '8회']

In [10]:
', '.join('13:00, 협녀, 10,000원'.split(', ')[1:-1])

'협녀'

In [31]:
def timetable_preprocessing(name):
    file_name = name.replace(' ', '%20').replace('(', '%28').replace(')', '%29')
    df = pd.read_csv(f'Theaters_merged_files/{file_name}.csv')

    # 변환 시작
    timetable_spread = list() # 리스트

    df_dict = df.T.to_dict()

    for i in df_dict: # row 를 dict 로, 컬럼도 추려서

        row = df_dict[i]
        # 컬럼명에서 따오기
        for column in list(df.columns)[2:-1]:
            if pd.isna(row[f'{column}']):
                continue

            detail = row[f'{column}'].split(', ')
            
            # 튜플이나 딕셔너리
            data = dict()
            
            # 가격 있고 제목에 콤마 -> 4 ~ 5
            # 가격 있고 -> 3
            if detail[-1][-2:] == '0원':
                data['영화명'] = ', '.join(detail[1:-1])
                data['가격'] = detail[-1]
            # 가격 없고 제목에 콤마 -> 3 ~ 4
            # 가격 없고 -> 2
            else :
                data['영화명'] = ', '.join(detail[1:])
                data['가격'] = np.nan
                
            data['상영관'] = row['상영관']
            data['상영일자'] = row['상영일자']
            data['상영시각'] = detail[0].strip()
            data['좌석수'] = row['좌석수']

            # 리스트에 append
            timetable_spread.append(data)

#     print(f'*** {name} 테이블 변환 완료 ***')
    
    timetable_spread = pd.DataFrame(timetable_spread)
    timetable_spread['영화관'] = name
    
    # # 한국영화만 가져오기
    timetable_spread['상영분류'] = timetable_spread['영화명']
    timetable_spread['상영분류'] = timetable_spread['상영분류'].apply(lambda x: x.split('(')[1].replace(')', '') if len(x.split('(')) > 1 else np.nan)
    timetable_spread['영화명'] = timetable_spread['영화명'].apply(lambda x: x.strip().split('(')[0])
    
#     k_index = list(timetable_spread[timetable_spread.isin({'영화명': k_movie_list})]['영화명'].dropna().index)
#     timetable_spread = timetable_spread.iloc[k_index]
    return timetable_spread[timetable_spread.isin({'영화명': k_movie_list})['영화명'] == True]
    
#     return timetable_spread # concat 을 위해 반환
#     timetable_spread.to_csv(f'Theaters_merged_files/{name}_timetable_spread.csv', index=False)

In [50]:
# 위에서 지정해준 인덱스 숫자 n과 m을 사용해 code['영화상영관명'][n:m] 으로 변경

data = pd.DataFrame() # concat 을 위해 미리 dataframe 만듦 # 중간에 다시 시작할 때는 빼고 돌리기
# for name in theater_list[:150]: 
# for name in theater_list[4:150]: # 1 CGV 울산신천 2 CGV 오산중앙 3 롯데시네마 대전관저 다음부터
# for name in theater_list[10:150]: # 9 메가박스 성수점 다음부터 
# for name in theater_list[73:150]: # 72 포천자동차극장 다음부터
# for name in theater_list[150:300]:
# for name in theater_list[193:300]: # 192 영화의전당 다음부터
# for name in theater_list[216:300]: # 215 롯데시네마 해운대 다음부터
# for name in theater_list[236:300]: # 233 CGV 여수웅천 234 CGV 순천, 235 CGV 목포평화광장 다음부터
# for name in theater_list[291:300]: # 290 이봄씨어터 다음부터
# for name in theater_list[300:450]: # 다영님 merge 중
# for name in theater_list[306:450]: # 305 메가박스 영종 다음부터
# for name in theater_list[306:450]: #  다음부터
for name in theater_list[450:]: 
# for name in theater_list[481:]: # 480 강릉자동차극장 다음부터
# for name in theater_list[484:]: # 483 여수자동차극장 다음부터
# for name in theater_list[495:]: # 491 용봉산시네마자동차극장, 492 무비마운틴자동차극장 493, 494, 495 다음부터 
# for name in theater_list[504:]: # 503 양평자동차극장 다음부터 
# for name in theater_list[514:]: # 513 양평시네마 다음부터 
# for name in theater_list[520:]: # 519 롯데시네마 평촌_샤롯데 다음부터 
# for name in theater_list[525:]: # 523 롯데시네마 마산터미널, 524로드시네마자동차극장 다음부터 
# for name in theater_list[527:]: # 526 블루마씨네자동차극장 다음부터 

# null 값 있었던 파일만 다시 concat
# cgv 압구정 165 
# cgv 신촌아트레온, 15 
# 롯데시네마 광주(롯데백화점), 432
# 롯데시네마 광주(롯데백화점)_샤롯데 461
# for name in theater_list[461:462]:
    try:
        print(name, '처리 시도 중...')
        timetable_spread = timetable_preprocessing(name) # concat
#         timetable_preprocessing(name) # 저장할 때 

        data = pd.concat([data, timetable_spread]) # concat 
    
#         print(name, '완료')
    except ValueError as e:
        print('[Error]', e)
print('*** 완료!!! ***')

씨네큐 부평 처리 시도 중...
완주 휴 시네마 처리 시도 중...
CGV 안성 처리 시도 중...
부산예술관 처리 시도 중...
롯데시네마 프리미엄칠곡 처리 시도 중...
롯데시네마 광주광산 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xb7 in position 22: invalid start byte
롯데시네마 프리미엄 진주 처리 시도 중...
롯데시네마 충장로 처리 시도 중...
롯데시네마 강동 처리 시도 중...
롯데시네마 월드타워_샤롯데 처리 시도 중...
메가박스 여수 처리 시도 중...
롯데시네마 광주(롯데백화점)_샤롯데 처리 시도 중...
롯데시네마 김포공항_샤롯데 처리 시도 중...
금성시네마 처리 시도 중...
CGV 남포 처리 시도 중...
CGV 칠곡 처리 시도 중...
논산시네마 처리 시도 중...
Cinema상상마당 처리 시도 중...
대구독립영화전용관 오오극장 처리 시도 중...
애관극장(인천) 처리 시도 중...
광주극장 처리 시도 중...
CGV 춘천명동 처리 시도 중...
CGV 광명철산 처리 시도 중...
명화극장 처리 시도 중...
롯데시네마 양주고읍 처리 시도 중...
롯데시네마 서귀포 처리 시도 중...
CGV 거제 처리 시도 중...
CGV 경기광주 처리 시도 중...
CGV 인천연수 처리 시도 중...
CGV 통영 처리 시도 중...
강릉자동차극장 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xb0 in position 22: invalid start byte
CGV 대구아카데미 처리 시도 중...
롯데시네마 에비뉴엘(명동)_샤롯데 처리 시도 중...
여수자동차극장 처리 시도 중...
[Error] 'utf-8' codec can't decode byte 0xbf in position 22: invalid start byte
화성시문화재단 모두누림센터 누림아트홀 처리 시도 중...
[Error] 'utf-8' codec 

In [52]:
print(data.shape)
print(len(data['영화관'].unique()))
data.sample(5)

(1659550, 8)
85


,가격,상영관,상영시각,상영일자,영화명,좌석수,영화관,상영분류
45495,"9,000원",01관,17:50,2018-04-19,나를 기억해,126.000000,롯데시네마 프리미엄칠곡,디지털
40967,"9,000원",02관,24:30,2017-01-24,더 킹,104.000000,CGV 동백,디지털
31405,"9,000원",01관,14:20,2016-12-08,미씽: 사라진 여자,158.000000,메가박스 공주,디지털
8717,"10,000원",03관,19:40,2015-06-05,은밀한 유혹,180.000000,롯데시네마 강동,디지털
19367,"6,000원",02관,23:20,2017-06-09,악녀,94.000000,메가박스 남원,디지털


In [53]:
data.loc[data['영화명'] == '협녀, 칼의 기억'].sample()

,가격,상영관,상영시각,상영일자,영화명,좌석수,영화관,상영분류
5036,"7,000원",03관,16:00,2015-08-15,"협녀, 칼의 기억",38.000000,논산시네마,디지털


In [54]:
# data.to_csv('Theaters_merged_files/show_timetable_0-150.csv', index=False) # 완료
# data.to_csv('Theaters_merged_files/show_timetable_150-300.csv', index=False) # 300만개
# data.to_csv('Theaters_merged_files/show_timetable_300-450.csv', index=False)
data.to_csv('Theaters_merged_files/show_timetable_450-.csv', index=False)
# data.to_csv('Theaters_merged_files/show_timetable_seat_error_modified.csv', index=False)

In [ ]:
# 에러

# 파일 없음
# CGV 청주(서문), CGV 용산아이파크몰, CGV 송파, 
# CGV 영등포
# 롯데시네마 광주광산, 화성시문화재단 모두누림센터 누림아트홀

# 데이터 없음
# 1 CGV 울산신천, 2 CGV 오산중앙, 3 롯데시네마 대전관저, 9 메가박스 성수점, 72 포천자동차극장,
# 192 영화의 전당, 215 롯데시네마 해운대, 233 CGV 여수웅천, 234 CGV 순천, 235 CGV 목포평화광장, 290 이봄씨어터,  


In [75]:
k_data = data.copy()

In [80]:
test = data[:10]

In [95]:
'연평해전' in k_movie_list

True

In [107]:
test

test['상영분류'] = test['영화명']
test['상영분류'] = test['상영분류'].apply(lambda x: x.split('(')[1].replace(')', '') if len(x.split('(')) > 1 else np.nan)
test['영화명'] = test['영화명'].apply(lambda x: x.strip().split('(')[0])
test

,가격,상영관,상영시각,상영일자,영화명,좌석수,영화관,상영분류
0,NaN,01관,13:20,2015-08-19,매드 맥스: 분노의 도로,206.000000,CGV 판교,nan
1,NaN,02관(SCREENX),14:50,2015-08-19,차이나타운,214.000000,CGV 판교,nan
2,NaN,03관,12:20,2015-08-19,뷰티 인사이드,203.000000,CGV 판교,nan
3,NaN,03관,15:20,2015-08-19,치외법권,203.000000,CGV 판교,nan
4,NaN,04관,13:50,2015-08-19,극비수사,180.000000,CGV 판교,nan
5,NaN,04관,16:20,2015-08-19,연평해전,180.000000,CGV 판교,nan
6,NaN,IMAX,11:50,2015-08-19,인터스텔라,297.000000,CGV 판교,nan
7,NaN,4DX,12:50,2015-08-19,샌 안드레아스,148.000000,CGV 판교,nan
8,NaN,4DX,15:50,2015-08-19,매드 맥스: 분노의 도로,148.000000,CGV 판교,nan
9,NaN,07관,14:20,2015-08-19,퇴마: 무녀굴,184.000000,CGV 판교,nan


In [122]:
k_index = list(test[test.isin({'영화명': k_movie_list})]['영화명'].dropna().index)
test.iloc[k_index]
# test[test.isin({'영화명': k_movie_list})]

,가격,상영관,상영시각,상영일자,영화명,좌석수,영화관,상영분류
1,NaN,02관(SCREENX),14:50,2015-08-19,차이나타운,214.000000,CGV 판교,nan
2,NaN,03관,12:20,2015-08-19,뷰티 인사이드,203.000000,CGV 판교,nan
3,NaN,03관,15:20,2015-08-19,치외법권,203.000000,CGV 판교,nan
4,NaN,04관,13:50,2015-08-19,극비수사,180.000000,CGV 판교,nan
5,NaN,04관,16:20,2015-08-19,연평해전,180.000000,CGV 판교,nan
9,NaN,07관,14:20,2015-08-19,퇴마: 무녀굴,184.000000,CGV 판교,nan


In [124]:
# 한국영화만 남기기
# k_data['상영분류'] = k_data['영화명']
# k_data['상영분류'] = k_data['상영분류'].apply(lambda x: x.split('(')[1].replace(')', '') if len(x.split('(')) > 1 else np.nan)
# k_data['영화명'] = k_data['영화명'].apply(lambda x: x.strip().split('(')[0])

# k_index = list(k_data[k_data.isin({'영화명': k_movie_list})]['영화명'].dropna().index)

k_data = k_data.iloc[k_index]

print(k_data.shape)
k_data.sample()

(6601770, 8)


,가격,상영관,상영시각,상영일자,영화명,좌석수,영화관,상영분류
12458,"17,000원",IMAX,12:50,2016-04-28,캡틴 아메리카: 시빌 워,297.000000,CGV 판교,IMAX 3D


# 전체 concat

In [55]:
df1 = pd.read_csv('Theaters_merged_files/show_timetable_0-150.csv').drop_duplicates()
df2 = pd.read_csv('Theaters_merged_files/show_timetable_150-300.csv').drop_duplicates()
df3 = pd.read_csv('Theaters_merged_files/show_timetable_300-450.csv').drop_duplicates()
df4 = pd.read_csv('Theaters_merged_files/show_timetable_450-.csv').drop_duplicates()

In [56]:
df = pd.concat([df1, df2, df3, df4])
print(df.shape)
df.sample()

(11874284, 8)


,가격,상영관,상영시각,상영일자,영화명,좌석수,영화관,상영분류
1992761,"10,000원",01관,15:10,2019-03-25,돈,169.000000,CGV광양,디지털


In [57]:
df.isnull().sum()

가격      468915
상영관          0
상영시각         0
상영일자         0
영화명          0
좌석수       7183
영화관          0
상영분류         0
dtype: int64

In [58]:
df.loc[df['좌석수'].isnull(), '영화관'].value_counts()

서울극장                   3295
CGV 산본                 2306
롯데시네마 안산고잔              923
메가박스 센트럴                187
CGV 명동역 씨네라이브러리         177
조이앤시네마 전주점              164
CGV 안산                   61
롯데시네마 광복                 46
롯데시네마 광주(롯데백화점)_샤롯데      24
Name: 영화관, dtype: int64

In [59]:
df.to_csv('Theaters_merged_files/show_timetable.csv', index=False)

# error 다시 concat

In [4]:
df = pd.read_csv('Theaters_merged_files/show_timetable.csv')
print(df.shape)
df.sample()

(11671656, 8)


,가격,상영관,상영시각,상영일자,영화명,좌석수,영화관,상영분류
8349063,"11,000원",04관,19:30,2016-11-05,럭키,155.000000,롯데시네마 울산성남,디지털


In [5]:
errored_list = ['CGV 압구정', 'CGV 신촌아트레온', '롯데시네마 광주(롯데백화점)', '롯데시네마 광주(롯데백화점)_샤롯데']

In [13]:
len(df[df.isin({'영화관': errored_list})['영화관'] == False])

# error_index = list(df.isin({'영화관': errored_list})['영화관'].dropna().index)
# len(error_index)
# df = df.drop(error_index)

# k_index = list(timetable_spread[timetable_spread.isin({'영화명': k_movie_list})]['영화명'].dropna().index)
# timetable_spread = timetable_spread.iloc[k_index]

11559251

In [15]:
data = pd.read_csv('Theaters_merged_files/show_timetable_seat_error_modified.csv')
data.shape

(112405, 8)

In [16]:
11559251+112405

11671656

In [17]:
pd.concat([df[df.isin({'영화관': errored_list})['영화관'] == False], data]).to_csv('Theaters_merged_files/show_timetable.csv', index=False)

In [ ]:
# concat 후에 할 작업
# 영화명 뒤에 붙어있는 (디지털), (IMAX) 등 따로 떼기
#     timetable_spread['상영분류'] = timetable_spread['영화명']
#     timetable_spread['상영분류'] = timetable_spread['상영분류'].apply(lambda x: x.split('(')[1].replace(')', '') if len(x.split('(')) > 1 else np.nan)
#     timetable_spread['영화명'] = timetable_spread['영화명'].apply(lambda x: x.split('(')[0])

    # 상영일자에서 요일 정보 컬럼 만들기 -> 주중(월화수목) 주말(금토일) 로 나누기
    timetable_spread['상영일자'] = pd.to_datetime(timetable_spread['상영일자'])
    timetable_spread['year'] = timetable_spread['상영일자'].dt.year
    timetable_spread['month'] = timetable_spread['상영일자'].dt.month
    timetable_spread['day'] = timetable_spread['상영일자'].dt.day
    timetable_spread['dayofweek'] = timetable_spread['상영일자'].dt.dayofweek

    timetable_spread['주말'] = 0
    # timetable_spread.loc[timetable_spread['dayofweek'] == 4, '주말'] = 1 # 금요일도 주중이니까 시간대 분류를 위해
    timetable_spread.loc[timetable_spread['dayofweek'] == 5, '주말'] = 1 
    timetable_spread.loc[timetable_spread['dayofweek'] == 6, '주말'] = 1 

    timetable_spread[['상영일자', 'year', 'month', 'day', 'dayofweek', '주말']]
    
    # 상영시각 등급
    timetable_spread.loc[timetable_spread['상영시각'] > '24:00', '상영시각등급'] = '나이트'
    timetable_spread.loc[timetable_spread['상영시각'] <= '24:00', '상영시각등급'] = '문라이트'
    timetable_spread.loc[timetable_spread['상영시각'] <= '22:00', '상영시각등급'] = '프라임'
    timetable_spread.loc[timetable_spread['상영시각'] <= '16:00', '상영시각등급'] = '데이라이트'
    timetable_spread.loc[timetable_spread['상영시각'] <= '13:00', '상영시각등급'] = '브런치'
    timetable_spread.loc[timetable_spread['상영시각'] <= '10:00', '상영시각등급'] = '모닝'
